In [5]:
import pickle
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torchvision
import matplotlib.pyplot as plt
import sinabs
import sinabs.activation
import sinabs.layers as sl
from sinabs.from_torch import from_model
import os
import shutil
from sklearn.model_selection import KFold
print(torch.cuda.is_available())
torch.cuda.get_device_name(0)
torch.cuda.empty_cache()

True


Data

In [6]:
class MyData(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.data_path = os.listdir(self.root_dir)
    
    def __getitem__(self, idx):
        data_name = self.data_path[idx]
        data_item_path = os.path.join(self.root_dir, data_name)
        with open(data_item_path, 'rb') as f:
            data = np.load(f)

        # ############## 测试压缩后的性能 ##############
        # data_zeros = np.zeros((100, 350))
        # data_zeros += data[:, ::2] + data[:, 1::2]
        # data = data_zeros
        # #############################################

        data = torch.from_numpy(data).float()
        data = torch.transpose(data, 0, 1)

        if data_name[-6] == '_':
            label = torch.tensor(eval(data_name[-5]), dtype=torch.long)
        else:
            label = torch.tensor(eval(data_name[-6:-4]), dtype=torch.long)
        
        return data, label

    def __len__(self):
        return len(self.data_path)

root_dir_1 = 'F:\Files\PhD/Braille\Data/braille-27letters-sphere/effect-speed/speed-v150-depth2.5/compress-test/compresstime-350/train'
root_dir_2 = 'F:\Files\PhD/Braille\Data/braille-27letters-sphere/effect-speed/speed-v150-depth2.5/compress-test/compresstime-350/test'

train_data = MyData(root_dir_1)
test_data  = MyData(root_dir_2)

In [7]:
print(train_data[0][0].shape)
print(len(train_data), len(test_data))

torch.Size([350, 100])
2160 1080


Training

In [8]:
lr = 1e-4

device = 'cuda:0'
k_folds = 6
kfold = KFold(n_splits=k_folds, shuffle=False)
batch_size = 81
loss_fn = nn.CrossEntropyLoss()

fold_train_acc = []
fold_val_acc = []

for fold, (train_ids, val_ids) in enumerate(kfold.split(train_data)):
    # 模型初始化
    linear_model = nn.Sequential(
        # nn.Linear(100, 200),
        # nn.ReLU(),
        # nn.Linear(200, 27)
        nn.Linear(100, 400),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(400, 600),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(600, 200),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(200, 27)
    )
    linear_model = from_model(linear_model, batch_size=81, input_shape=(1, 350, 100), 
                              add_spiking_output=True).to(device)
    optimizer = torch.optim.Adam(linear_model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    # 分割数据集
    train_sub = Subset(train_data, train_ids)
    val_sub = Subset(train_data, val_ids)
    # 创建数据加载器
    train_loader = DataLoader(train_sub, batch_size, shuffle=True, drop_last=True)
    val_loader  = DataLoader(val_sub, batch_size, drop_last=True)

    # 训练模型
    linear_model.train()
    epochs = 40
    for e in range(epochs):
        running_loss = 0.
        acc = 0
        scheduler.step()
        for i, (input, target) in enumerate(train_loader):
            optimizer.zero_grad()
            linear_model.reset_states()

            input = input.to(device)
            target = target.to(device)
            output = linear_model(input)
            sum_output = output.sum(1)
            loss = loss_fn(sum_output, target)
            loss.backward()
            optimizer.step()

            running_loss += loss

            for j in range(batch_size):
                if sum_output[j].argmax() == target[j]:
                    acc += 1

        print("epoch: %d, accuracy: %.2f%%, running_loss: %.2f, current_lr: %.6f" 
              % (e, acc/len(train_sub)*100, running_loss, scheduler.get_last_lr()[0]) )
    fold_train_acc.append(np.around(acc/len(train_sub)*100, 2))

    # 验证模型
    acc_num = 0
    for i, (data, target) in enumerate(val_loader):
        with torch.no_grad():
            linear_model.reset_states()
            data = data.to(device)
            target = target.to(device)
            output = linear_model(data)
            sum_output = output.sum(1)

        for j in range(batch_size):
            if sum_output[j].argmax() == target[j]:
                acc_num += 1
    print("accuracy on validation set: %.2f%%" % (acc_num/len(val_sub)*100))
    fold_val_acc.append(np.around(acc_num/len(val_sub)*100, 2))

epoch: 0, accuracy: 4.56%, running_loss: 72.45, current_lr: 0.000100
epoch: 1, accuracy: 4.94%, running_loss: 72.22, current_lr: 0.000100
epoch: 2, accuracy: 7.11%, running_loss: 71.33, current_lr: 0.000100
epoch: 3, accuracy: 14.00%, running_loss: 67.99, current_lr: 0.000100
epoch: 4, accuracy: 21.56%, running_loss: 62.03, current_lr: 0.000100
epoch: 5, accuracy: 29.17%, running_loss: 54.43, current_lr: 0.000100
epoch: 6, accuracy: 40.94%, running_loss: 44.03, current_lr: 0.000100
epoch: 7, accuracy: 51.56%, running_loss: 34.08, current_lr: 0.000100
epoch: 8, accuracy: 61.33%, running_loss: 26.68, current_lr: 0.000100
epoch: 9, accuracy: 67.94%, running_loss: 21.48, current_lr: 0.000100
epoch: 10, accuracy: 73.83%, running_loss: 17.11, current_lr: 0.000100
epoch: 11, accuracy: 75.44%, running_loss: 16.09, current_lr: 0.000100
epoch: 12, accuracy: 81.39%, running_loss: 10.71, current_lr: 0.000100
epoch: 13, accuracy: 84.67%, running_loss: 9.08, current_lr: 0.000100
epoch: 14, accuracy:

In [9]:
print(fold_train_acc)
print(fold_val_acc)

sum = 0
cnt = 0
for i in fold_train_acc:
    sum += i
    cnt += 1
print(sum/cnt)

sum = 0
cnt = 0
for i in fold_val_acc:
    sum += i
    cnt += 1
print(sum/cnt)

[93.83, 96.83, 95.78, 96.22, 96.72, 95.72]
[86.11, 88.33, 86.11, 86.67, 88.33, 85.83]
95.85000000000001
86.89666666666666


In [10]:
model_path = './models/speed150-ct350.pth'
torch.save(linear_model, model_path)

Testing

In [11]:
model = torch.load(model_path)
test_loader  = DataLoader(test_data, batch_size, drop_last=True)

acc_num = 0
for i, (data, target) in enumerate(test_loader):
    with torch.no_grad():
        model.reset_states()
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        sum_output = output.sum(1)

    for j in range(batch_size):
        if sum_output[j].argmax() == target[j]:
            acc_num += 1
print("accuracy on testing set: %.2f%%" % (acc_num/len(test_data)*100))

accuracy on testing set: 93.70%
